---
title: Logistic Regression
jupyter: python3
---

## Introduction

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tools4ds/DS701-Course-Notes/blob/main/ds701_book/jupyter_notebooks/18-Regression-II-Logistic.ipynb)

In [ ]:
#| echo: false
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib as mp
import sklearn
from IPython.display import Image, HTML
import statsmodels.api as sm
from sklearn import model_selection
from sklearn import metrics

import laUtilities as ut

%matplotlib inline

So far we have seen linear regression: 

* a continuous valued observation is estimated as a linear (or affine) function
 of the independent variables.

Now we will look at the following situation.

## Estimating a Probability

::: {.incremental}

* Imagine that you are observing a binary variable -- value 0 or 1.

* That is, these could be pass/fail, admit/reject, Democrat/Republican, etc.

* Assume there is some __probability__ of observing a 1, and that probability is a
function of certain independent variables.

* So the key properties of a problem that make it appropriate for logistic
regression are:

    * You are trying to predict a __categorical__ variable
    * You want to estimate a __probability__ of seeing a particular value of the
      categorical variable.

:::

## Example: Grad School Admission

::: {.content-visible when-profile="web"}

::: {.callout-note}
The following example was adapted from this 
[URL](http://www.ats.ucla.edu/stat/r/dae/logit.htm) which seems to be no longer
available. There is an 
[archive of the page](https://web.archive.org/web/20161118221128/http://www.ats.ucla.edu/stat/r/dae/logit.htm)
and an [archive of the dataset](https://web.archive.org/web/20161022051727/http://www.ats.ucla.edu/stat/data/binary.csv).
:::
:::

Let's consider this question:

> What is the probability I will be admitted to Grad School?

Let's see how variables, such as,

* _GRE_ (Graduate Record Exam scores), 
* _GPA_ (grade point average), and 
* prestige of the undergraduate institution

affect admission into graduate school. 

::: {.content-visible when-profile="slides"}
## Example continued
:::

The response variable, admit/don't admit, is a binary variable.

So there are three predictor variables: __gre,__ __gpa__ and __rank.__ 

* We will treat the variables _gre_ and _gpa_ as continuous. 
* The variable _rank_ takes on the values 1 through 4 with 1 being the highest prestige.

::: {.content-visible when-profile="slides"}
## Example continued
:::

Let's look at 10 lines of the data:

In [ ]:
#| echo: false
# original data source: http://www.ats.ucla.edu/stat/data/binary.csv
df = pd.read_csv('data/ats-admissions.csv') 
df.head(10)

In [ ]:
df.shape

::: {.content-visible when-profile="slides"}
## Example continued
:::

and some summary statistics:

In [ ]:
#| code-fold: true
df.describe()

::: {.content-visible when-profile="slides"}
## Example continued
:::

We can also plot histograms of the variables:

In [ ]:
#| code-fold: true
df.hist(figsize = (10, 4));

> Note how `df.hist()` automatically plots a histogram for each column in the
> dataframe as a subplot.

::: {.content-visible when-profile="slides"}
## Example continued
:::

Let's look at how each independent variable affects admission probability by
plotting the mean admission probability as a function of the independent variable.

> Note that there's a greatly expanded [`groupby`](https://tools4ds.github.io/DS701-Course-Notes/02B-Pandas.html#understanding-pandas-dataframe.groupby) section in the Pandas refresher.

We add error bars to the means to indicate the standard error of the mean.

::: {.content-visible when-profile="slides"}
## Example continued
:::

First, __rank__:

In [ ]:
#| code-fold: true
import numpy as np

# Calculate mean and standard error
grouped = df.groupby('rank')['admit']
means = grouped.mean()

# Compute 'standard error of the mean'
errors = grouped.std() / np.sqrt(grouped.count())

# Plot with error bars
ax = means.plot(marker='o', yerr=errors, fontsize=12, capsize=5)
ax.set_ylabel('P[admit]', fontsize=16)
ax.set_xlabel('Rank', fontsize=16);

::: {.content-visible when-profile="slides"}
## Example continued
:::

Next, __GRE__:

In [ ]:
#| code-fold: true
grouped_gre = df.groupby('gre')['admit']
means_gre = grouped_gre.mean()
errors_gre = grouped_gre.std() / np.sqrt(grouped_gre.count())

ax = means_gre.plot(marker='o', yerr=errors_gre, fontsize=12, capsize=5)
ax.set_ylabel('P[admit]', fontsize=16)
ax.set_xlabel('GRE', fontsize=16);

::: {.content-visible when-profile="slides"}
## Example continued
:::

Finally, __GPA__ (for this visualization, we aggregate GPA into 8 bins):

In [ ]:
#| code-fold: true
bins = np.linspace(df.gpa.min(), df.gpa.max(), 8)
bin_centers = (bins[:-1] + bins[1:]) / 2
grouped_gpa = df.groupby(np.digitize(df.gpa, bins)).mean()['admit']
ax = grouped_gpa.plot(marker='o', fontsize=12)
ax.set_ylabel('P[admit]', fontsize=16)
ax.set_xlabel('GPA', fontsize=16)
ax.set_xticks(range(1, len(bin_centers) + 1))
ax.set_xticklabels([f'{center:.2f}' for center in bin_centers], rotation=45);

::: {.content-visible when-profile="slides"}
## Example continued
:::

Finally, we plot admission status versus GRE score for each data point for each of the four ranks:

In [ ]:
#| code-fold: true
df1 = df[df['rank']==1]
df2 = df[df['rank']==2]
df3 = df[df['rank']==3]
df4 = df[df['rank']==4]

fig = plt.figure(figsize = (10, 5))

ax1 = fig.add_subplot(221)
df1.plot.scatter('gre','admit', ax = ax1)
plt.title('Rank 1 Institutions')

ax2 = fig.add_subplot(222)
df2.plot.scatter('gre','admit', ax = ax2)
plt.title('Rank 2 Institutions')

ax3 = fig.add_subplot(223, sharex = ax1)
df3.plot.scatter('gre','admit', ax = ax3)
plt.title('Rank 3 Institutions')

ax4 = fig.add_subplot(224, sharex = ax2)
plt.title('Rank 4 Institutions')
df4.plot.scatter('gre','admit', ax = ax4);

What we want to do is to fit a model that predicts the probability of admission
as a function of these independent variables.

## Logistic Regression

::: {.incremental}
* Logistic regression is concerned with estimating a __probability.__

* However, all that is available are categorical observations, which we will code as 0/1.

* That is, these could be pass/fail, admit/reject, Democrat/Republican, etc.

* Now, a linear function like $\beta_0 + \beta_1 x$ cannot be used to predict
probability directly, because 
    * the linear function takes on all values (from -$\infty$ to +$\infty$), 
    * and probability only ranges over $[0, 1]$.

:::

## Odds and Log-Odds

However, there is a transformation of probability that works: it is called
__log-odds__.

For any probabilty $p$, the __odds__ is defined as $p/(1-p)$, which is the ratio
of the probability of an event to the probability of the non-event.

Notice that odds vary from 0 to $\infty$, and odds < 1 indicates that $p < 1/2$.

Now, there is a good argument that to fit a linear function, instead of using
odds, we should use log-odds. 

::: {.content-visible when-profile="slides"}
## Odds and Log-Odds
:::

That is simply $\log p/(1-p)$ which is also called the __logit__ function, which
is an abbreviation for **log**istic un**it**.

In [ ]:
#| code-fold: true
pvec = np.linspace(0.01, 0.99, 100)
ax = plt.figure(figsize = (6, 4)).add_subplot()
ax.plot(pvec, np.log(pvec / (1-pvec)))
ax.tick_params(labelsize=12)
ax.set_xlabel('Probability', fontsize = 14)
ax.set_ylabel('Log-Odds', fontsize = 14)
ax.set_title('Logit Function: $\log (p/1-p)$', fontsize = 16);

::: {.content-visible when-profile="slides"}
## Odds and Log-Odds
:::

So, logistic regression does the following: it does a linear regression of
$\beta_0 + \beta_1 x$ against $\log p/(1-p)$.

That is, it fits:

$$
\begin{aligned}
\beta_0 + \beta_1 x &= \log \frac{p(x)}{1-p(x)} \\
e^{\beta_0 + \beta_1 x} &= \frac{p(x)}{1-p(x)} \quad \text{(exponentiate both sides)} \\
e^{\beta_0 + \beta_1 x} (1-p(x)) &= p(x) \quad \text{(multiply both sides by $1-p(x)$)} \\
e^{\beta_0 + \beta_1 x}  &= p(x) + p(x)e^{\beta_0 + \beta_1 x} \quad \text{(distribute $p(x)$)} \\
\frac{e^{\beta_0 + \beta_1 x}}{1 +e^{\beta_0 + \beta_1 x}} &= p(x)
\end{aligned}
$$

::: {.content-visible when-profile="slides"}
## Odds and Log-Odds
:::

So, logistic regression fits a probability of the following form:

$$
p(x) = P(y=1\mid x) = \frac{e^{\beta_0+\beta_1 x}}{1+e^{\beta_0+\beta_1 x}}.
$$

This is a **sigmoid function**; when $\beta_1 > 0$, 

* as $x\rightarrow \infty$, then $p(x)\rightarrow 1$ and 
* as $x\rightarrow -\infty$, then $p(x)\rightarrow 0$.

::: {.content-visible when-profile="slides"}
## Odds and Log-Odds
:::

In [ ]:
#| code-fold: true
alphas = [-4, -8,-12,-20]
betas = [0.4,0.4,0.6,1]
x = np.arange(40)
fig = plt.figure(figsize=(8, 6)) 
ax = plt.subplot(111)

for i in range(len(alphas)):
    a = alphas[i]
    b = betas[i]
    y = np.exp(a+b*x)/(1+np.exp(a+b*x))
#     plt.plot(x,y,label=r"$\frac{e^{%d + %3.1fx}}{1+e^{%d + %3.1fx}}\;\beta_0=%d, \beta_1=%3.1f$" % (a,b,a,b,a,b))
    ax.plot(x,y,label=r"$\beta_0=%d,$    $\beta_1=%3.1f$" % (a,b))
ax.tick_params(labelsize=12)
ax.set_xlabel('x', fontsize = 14)
ax.set_ylabel('$p(x)$', fontsize = 14)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), prop={'size': 16})
ax.set_title('Logistic Functions', fontsize = 16);

Parameter $\beta_1$ controls how fast $p(x)$ raises from $0$ to $1$

The value of -$\beta_0$/$\beta_1$ shows the value of $x$ for which $p(x)=0.5$

::: {.content-visible when-profile="slides"}
## Odds and Log-Odds
:::

Another interpretation of $\beta_0$ is that it gives the __base rate__ -- the
unconditional probability of a 1.   

That is, if you knew nothing about a
particular data item, then $p(x) = 1/(1+e^{-\beta_0})$.

In [ ]:
#| fig-align: center

# plot the base rate as a function of beta_0
beta_0_values = np.linspace(-10, 10, 100)
base_rate = 1 / (1 + np.exp(-beta_0_values))
plt.figure(figsize=(6, 4))
plt.plot(beta_0_values, base_rate)
plt.xlabel('beta_0')
plt.ylabel('Base Rate')
plt.title('Base Rate as a function of beta_0')
plt.show()

::: {.content-visible when-profile="slides"}
## Odds and Log-Odds
:::

The function $f(x) = \log (x/(1-x))$ is called the __logit__ function.

So a compact way to describe logistic regression is that it finds regression
coefficients $\beta_0, \beta_1$ to fit:

$$
\text{logit}\left(p(x)\right)=\log\left(\frac{p(x)}{1-p(x)} \right) = \beta_0 + \beta_1 x.
$$

Note also that the __inverse__ logit function is:

$$
\text{logit}^{-1}(x) = \frac{e^x}{1 + e^x}.
$$

Somewhat confusingly, this is called the __logistic__ function.

::: {.content-visible when-profile="slides"}
## Odds and Log-Odds
:::

So, the best way to think of logistic regression is that we compute a linear function:
    
$$
\beta_0 + \beta_1 x,
$$
    
and then *map* that to a probability using the inverse $\text{logit}$ function:

$$
\frac{e^{\beta_0+\beta_1 x}}{1+e^{\beta_0+\beta_1 x}}.
$$

## Logistic vs Linear Regression

Let's take a moment to compare linear and logistic regression.

In __Linear regression__ we fit 

$$
y_i = \beta_0 +\beta_1 x_i + \epsilon_i.
$$

We do the fitting by minimizing the sum of squared errors $\Vert\epsilon\Vert$.
This can be done in closed form using either geometric arguments or by calculus.

Now, if $\epsilon_i$ comes from a normal distribution with mean zero and some
fixed variance, then minimizing the sum of squared errors is exactly the same as finding the
maximum likelihood of the data with respect to the probability of the errors.

So, in the case of linear regression, it is a lucky fact that the __MLE__ of
$\beta_0$ and $\beta_1$ can be found by a __closed-form__ calculation.

::: {.content-visible when-profile="slides"}
## Logistic vs Linear Regression
:::

In __Logistic regression__ we fit 

$$
\text{logit}(p(x_i)) = \beta_0 + \beta_1 x_i.
$$


with $\text{P}(y_i=1\mid x_i)=p(x_i).$

How should we choose parameters?   

Here too, we use Maximum Likelihood Estimation of the parameters.

That is, we choose the parameter values that maximize the likelihood of the data given the model.

$$
\text{P}(y_i \mid x_i) = 
\left\{\begin{array}{lr}\text{logit}^{-1}(\beta_0 + \beta_1 x_i)& \text{if } y_i = 1\\
1 - \text{logit}^{-1}(\beta_0 + \beta_1 x_i)& \text{if } y_i = 0\end{array}\right.
$$

::: {.content-visible when-profile="slides"}
## Logistic vs Linear Regression
:::

We can write this as a single expression:

$$
\text{P}(y_i \mid x_i) = \text{logit}^{-1}(\beta_0 + \beta_1 x_i)^{y_i} (1-\text{logit}^{-1}(\beta_0 + \beta_1 x_i))^{1-y_i}.
$$

We then use this to compute the __likelihood__ of parameters $\beta_0$, $\beta_1$:

$$
L(\beta_0, \beta_1 \mid x_i, y_i) = \text{logit}^{-1}(\beta_0 + \beta_1 x_i)^{y_i} (1-\text{logit}^{-1}(\beta_0 + \beta_1 x_i))^{1-y_i},
$$

which is a function that we can maximize, for example, with gradient descent.

## Logistic Regression In Practice

So, in summary, we have:

**Input** pairs $(x_i,y_i)$

**Output** parameters $\widehat{\beta_0}$ and $\widehat{\beta_1}$ that maximize the
likelihood of the data given these parameters for the logistic regression model.

**Method** Maximum likelihood estimation, obtained by gradient descent.

The standard package will give us a coefficient $\beta_i$ for each
independent variable (feature).

::: {.content-visible when-profile="slides"}
## Logistic Regression in Practice
:::

If we want to include a constant (i.e., $\beta_0$) we need to add a column of 1s (just
like in linear regression).

In [ ]:
#| code-fold: true
df['intercept'] = 1.0
train_cols = df.columns[1:]
train_cols

In [ ]:
#| code-fold: true
logit = sm.Logit(df['admit'], df[train_cols])
 
# fit the model
result = logit.fit() 

::: {.content-visible when-profile="slides"}
## Logistic Regression in Practice
:::

In [ ]:
#| code-fold: true
result.summary()

Notice that all of our independent variables are considered significant (no
confidence intervals contain zero).

## Using the Model

Note that by fitting a model to the data, we can make predictions for inputs that
were not in the training data.  

Furthermore, we can make a prediction of a probability for cases where we don't
have enough data to estimate the probability directly -- e.g., for specific
parameter values.

Let's see how well the model fits the data.

::: {.content-visible when-profile="slides"}
## Using the Model
:::

We have three independent variables, so in each case we'll use average values
for the two that we aren't evaluating.

GPA (GRE = 600, Rank = 2.5):

In [ ]:
#| code-fold: true
#| fig-align: center
bins = np.linspace(df.gpa.min(), df.gpa.max(), 10)
groups = df.groupby(np.digitize(df.gpa, bins))
prob = [result.predict([600, b, 2.5, 1.0]) for b in bins]
ax = plt.figure(figsize = (7, 4)).add_subplot()
ax.plot(bins, prob)
ax.plot(bins,groups.admit.mean(),'o')
ax.tick_params(labelsize=12)
ax.set_xlabel('gpa', fontsize = 14)
ax.set_ylabel('P[admit]', fontsize = 14)
ax.set_title('Marginal Effect of GPA', fontsize = 16);

::: {.content-visible when-profile="slides"}
## Logistic Regression in Practice
:::

GRE Score (GPA = 3.4, Rank = 2.5):

In [ ]:
#| code-fold: true
#| fig-align: center
prob = [result.predict([b, 3.4, 2.5, 1.0]) for b in sorted(df.gre.unique())]
ax = plt.figure(figsize = (7, 4)).add_subplot()
ax.plot(sorted(df.gre.unique()), prob)
ax.plot(df.groupby('gre').mean()['admit'],'o')
ax.tick_params(labelsize=12)
ax.set_xlabel('gre', fontsize = 14)
ax.set_ylabel('P[admit]', fontsize = 14)
ax.set_title('Marginal Effect of GRE', fontsize = 16);

::: {.content-visible when-profile="slides"}
## Logistic Regression in Practice
:::

Institution Rank (GRE = 600, GPA = 3.4):

In [ ]:
#| code-fold: true
#| fig-align: center
prob = [result.predict([600, 3.4, b, 1.0]) for b in range(1,5)]
ax = plt.figure(figsize = (7, 4)).add_subplot()
ax.plot(range(1,5), prob)
ax.plot(df.groupby('rank').mean()['admit'],'o')
ax.tick_params(labelsize=12)
ax.set_xlabel('Rank', fontsize = 14)
ax.set_xlim([0.5,4.5])
ax.set_ylabel('P[admit]', fontsize = 14)
ax.set_title('Marginal Effect of Rank', fontsize = 16);

## Logistic Regression in Perspective

At the start of lecture we emphasized that logistic regression is concerned with
estimating a __probability__ model for __discrete__ (0/1) data. 

However, it may well be the case that we want to do something with the
probability that amounts to __classification.__

For example, we may classify data items using a rule such as "Assign item $x_i$
to Class 1 if $p(x_i) > 0.5$".

For this reason, logistic regression could be considered a classification method.

::: {.content-visible when-profile="slides"}
## Logistic Regression in Perspective
:::

Let's use our logistic regression as a classifier.

We want to ask whether we can correctly predict whether a student gets admitted
to graduate school.

Let's separate our training and test data:

In [ ]:
#| code-fold: true
X_train, X_test, y_train, y_test = model_selection.train_test_split(
        df[train_cols], df['admit'],
        test_size=0.4, random_state=1)

::: {.content-visible when-profile="slides"}
## Logistic Regression in Perspective
:::

Now, there are some standard metrics used when evaluating a binary classifier.

Let's say our classifier is outputting "yes" when it thinks the student will be admitted.

There are four cases:

* Classifier says "yes", and student __is__ admitted:  __True Positive.__
* Classifier says "yes", and student __is not__ admitted:  __False Positive.__
* Classifier says "no", and student __is__ admitted:  __False Negative.__
* Classifier says "no", and student __is not__ admitted:  __True Negative.__

::: {.content-visible when-profile="slides"}
## Logistic Regression in Perspective
:::

__Precision__ is the fraction of "yes" classifications that are correct:

$$
\mbox{Precision} = \frac{\mbox{True Positives}}{\mbox{True Positives + False Positives}}.
$$
    
__Recall__ is the fraction of admits that we say "yes" to:

$$
\mbox{Recall} = \frac{\mbox{True Positives}}{\mbox{True Positives + False Negatives}}.
$$

In [ ]:
#| code-fold: true
def evaluate(y_train, X_train, y_test, X_test, threshold):

    # learn model on training data
    logit = sm.Logit(y_train, X_train)
    result = logit.fit(disp=False)
    
    # make probability predictions on test data
    y_pred = result.predict(X_test)
    
    # threshold probabilities to create classifications
    y_pred = y_pred > threshold
    
    # report metrics
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    return precision, recall

precision, recall = evaluate(y_train, X_train, y_test, X_test, 0.5)

print(f'Precision: {precision:0.3f}, Recall: {recall:0.3f}')

::: {.content-visible when-profile="slides"}
## Logistic Regression in Perspective
:::

Now, let's get a sense of average accuracy:

In [ ]:
#| code-fold: true
PR = []
for i in range(20):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(
            df[train_cols], df['admit'],
            test_size=0.4)
    PR.append(evaluate(y_train, X_train, y_test, X_test, 0.5))

In [ ]:
#| code-fold: true
avgPrec = np.mean([f[0] for f in PR])
avgRec = np.mean([f[1] for f in PR])
print(f'Average Precision: {avgPrec:0.3f}, Average Recall: {avgRec:0.3f}')

::: {.content-visible when-profile="slides"}
## Logistic Regression in Perspective
:::

Sometimes we would like a single value that describes the overall performance of
the classifier.

For this, we take the harmonic mean of precision and recall, called __F1 Score__:

$$
\mbox{F1 Score} = 2 \;\;\frac{\mbox{Precision} \cdot \mbox{Recall}}{\mbox{Precision} + \mbox{Recall}}.
$$

::: {.content-visible when-profile="slides"}
## Logistic Regression in Perspective
:::

Using this, we can evaluate other settings for the threshold.

In [ ]:
#| code-fold: true
import warnings
warnings.filterwarnings("ignore")
def evalThresh(df, thresh):
    PR = []
    for i in range(20):
        X_train, X_test, y_train, y_test = model_selection.train_test_split(
                df[train_cols], df['admit'],
                test_size=0.4)
        PR.append(evaluate(y_train, X_train, y_test, X_test, thresh))
    avgPrec = np.mean([f[0] for f in PR])
    avgRec = np.mean([f[1] for f in PR])
    return 2 * (avgPrec * avgRec) / (avgPrec + avgRec), avgPrec, avgRec

tvals = np.linspace(0.05, 0.8, 50)
f1vals = [evalThresh(df, tval)[0] for tval in tvals]

In [ ]:
#| code-fold: true
#| fig-align: center
plt.figure(figsize=(6, 4))
plt.plot(tvals,f1vals)
plt.ylabel('F1 Score')
plt.xlabel('Threshold for Classification')
plt.title('F1 as a function of Threshold');

Based on this plot, we can say that the best classification threshold appears to
be around 0.3, where precision and recall are:

In [ ]:
#| code-fold: true
F1, Prec, Rec = evalThresh(df, 0.3)
print('Best Precision: {:0.3f}, Best Recall: {:0.3f}'.format(Prec, Rec))

::: {.content-visible when-profile="web"}

The example here is based on
http://blog.yhathq.com/posts/logistic-regression-and-python.html
where you can find additional details.

:::

## Recap

* Logistic regression is used to predict a probability.
* It is a linear model for the log-odds.
* It is fit by maximum likelihood.
* It can be evaluated as a classifier.